In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# loading of dataset

In [ ]:
plant1_sensor = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv")
plant1_generation =pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv")
plant2_sensor = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv")
plant2_generation = pd.read_csv("/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv")

# Exploring dataset- count, columns, basic stats

In [ ]:
plant1_sensor.head()

In [ ]:
plant1_generation.head()

In [ ]:
plant2_sensor.head()

In [ ]:
plant2_generation.head()

In [ ]:
plant1_sensor.describe().T

In [ ]:
plant1_generation.describe().T

In [ ]:
plant2_sensor.describe().T

In [ ]:
plant2_generation.describe().T

In [ ]:
plant1_sensor.info()

In [ ]:
plant1_generation.info()

In [ ]:
plant2_sensor.info()

In [ ]:
plant2_generation.info()

In [ ]:
plant1_sensor.columns

In [ ]:
plant1_generation.columns

In [ ]:
plant2_sensor.columns

In [ ]:
plant2_generation.columns

# Check if any missing values

In [ ]:
plant1_sensor.isnull().sum()

In [ ]:
plant1_generation.isnull().sum()

In [ ]:
plant2_sensor.isnull().sum()

In [ ]:
plant2_generation.isnull().sum()

# In our dataset DATE_TIME is of object type. So we will convert it to datetime type

In [ ]:
from datetime import datetime

In [ ]:
plant1_generation['DATE'] = [datetime.strftime(i,format="%d-%m-%Y") for i in pd.to_datetime(plant1_generation.DATE_TIME)]
plant1_generation['DATE'] = pd.to_datetime(plant1_generation['DATE'])

In [ ]:
plant2_generation['DATE'] = [datetime.strftime(i,format="%Y-%m-%d") for i in pd.to_datetime(plant2_generation.DATE_TIME)]
plant2_generation['DATE'] = pd.to_datetime(plant2_generation['DATE'])

In [ ]:
plant1_sensor['DATE'] = [datetime.strftime(i,format="%Y-%m-%d") for i in pd.to_datetime(plant1_sensor.DATE_TIME)]
plant1_sensor['DATE'] = pd.to_datetime(plant1_sensor['DATE'])

In [ ]:
plant2_sensor['DATE'] = [datetime.strftime(i,format="%Y-%m-%d") for i in pd.to_datetime(plant2_sensor.DATE_TIME)]
plant2_sensor['DATE'] = pd.to_datetime(plant2_sensor['DATE'])

# Mean value of DAILY_YIELD of plant 1 & 2

In [ ]:
plant1_generation["DAILY_YIELD"].mean()

In [ ]:
plant2_generation["DAILY_YIELD"].mean()

In [ ]:
# finding the mean of both the generator
total_yeild = plant1_generation["DAILY_YIELD"].sum() + plant2_generation["DAILY_YIELD"].sum()
total_generator = len(plant1_generation["DAILY_YIELD"]) + len(plant2_generation["DAILY_YIELD"])
Mean_of_both= float(total_yeild)/float(total_generator)
print("Mean value of daily yeild of both the generators is : {}",format(Mean_of_both))

# Total IRRADIATION per day

In [ ]:
plant1_total_Irradiation = plant1_sensor.groupby(['DATE']).IRRADIATION.sum()
plant2_total_Irradiation = plant2_sensor.groupby(['DATE']).IRRADIATION.sum()

In [ ]:
pd.DataFrame({'Date':plant1_total_Irradiation.index,'Plant1_Daily_Irradiation':plant1_total_Irradiation.values,'Plant2_Daily_Irradiation':plant2_total_Irradiation.values})

# Max ambient and module temperature

In [ ]:
max_amb_mod_temp = pd.DataFrame({'Max Ambient Temp' : [plant1_sensor.AMBIENT_TEMPERATURE.max(), plant2_sensor.AMBIENT_TEMPERATURE.max()], 'Max Module Temp': [plant1_sensor.MODULE_TEMPERATURE.max(), plant2_sensor.MODULE_TEMPERATURE.max()]},index=['Plant_1','Plant_2'])
max_amb_mod_temp

# **How many inverters are there for each plant?**

In [ ]:
print("No. of inverters in Plant_1 : ",len(plant1_generation.SOURCE_KEY.value_counts()))
print("No. of inverters in Plant_2 : ",len(plant2_generation.SOURCE_KEY.value_counts()))

# Maximum/minimum amount of DC/AC Power generated in a time interval/day

# Plant1

In [ ]:
pd.DataFrame({'DC_Power_Max':plant1_generation.groupby(['DATE']).DC_POWER.max().values,'DC_Power_Min':plant1_generation.groupby(['DATE']).DC_POWER.min().values,
              'AC_Power_Max':plant1_generation.groupby(['DATE']).AC_POWER.max().values,'AC_Power_Min':plant1_generation.groupby(['DATE']).AC_POWER.min().values},
             index=plant1_generation.groupby(['DATE']).DC_POWER.max().index)

# Plant2

In [ ]:
pd.DataFrame({'DC_Power_Max':plant2_generation.groupby(['DATE']).DC_POWER.max().values,'DC_Power_Min':plant2_generation.groupby(['DATE']).DC_POWER.min().values,
              'AC_Power_Max':plant2_generation.groupby(['DATE']).AC_POWER.max().values,'AC_Power_Min':plant2_generation.groupby(['DATE']).AC_POWER.min().values},
             index=plant2_generation.groupby(['DATE']).DC_POWER.max().index)

# Inverter (source_key) that has produced maximum DC/AC power is

In [ ]:
dc_ac =pd.DataFrame({'Inverter_Source_Key':[plant1_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().sort_values(ascending=False).index[0],
                    plant2_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().sort_values(ascending=False).index[0]],
                    'Max_DC':[plant1_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().max(),plant2_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().max()],
                        'Max_AC':[plant1_generation.groupby(['SOURCE_KEY']).AC_POWER.sum().max(),plant2_generation.groupby(['SOURCE_KEY']).AC_POWER.sum().max()],
                     },index=['Plant_1','Plant_2'])
dc_ac

# Rank the inverters based on the DC/AC power they produce

# Plant_1

In [ ]:
index_dc_p1 = plant1_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().sort_values(ascending=False).index
index_ac_p1 = plant1_generation.groupby(['SOURCE_KEY']).AC_POWER.sum().sort_values(ascending=False).index

In [ ]:

ac_dict_p1 = dict() 
for i,j in zip(index_dc_p1,range(1,23,1)):
    ac_dict_p1[i] = j
ac_pr_per_dc_rank = []
for i in index_dc_p1:
    ac_pr_per_dc_rank.append(ac_dict_p1[i])
ac_pr_per_dc_rank

In [ ]:
plant1_dc_ac_rank = pd.DataFrame({'Plant1_Inverter_Source_Key':index_dc_p1,'DC_Production_Rank':range(1,23,1),'AC_Production_Rank':ac_pr_per_dc_rank})
plant1_dc_ac_rank

Plant_2

In [ ]:
index_dc_p2 = plant2_generation.groupby(['SOURCE_KEY']).DC_POWER.sum().sort_values(ascending=False).index
index_ac_p2 = plant2_generation.groupby(['SOURCE_KEY']).AC_POWER.sum().sort_values(ascending=False).index

In [ ]:
ac_dict_p2 = dict()
for i,j in zip(index_dc_p2,range(1,23,1)):
    ac_dict_p2[i] = j
ac_pr_per_dc_rank_p2 = []
for i in index_dc_p2:
    ac_pr_per_dc_rank_p2.append(ac_dict_p2[i])
ac_pr_per_dc_rank_p2

In [ ]:
plant_2_dc_ac_rank = pd.DataFrame({'Plant_2_Inverter_Source_Key':index_dc_p2,'DC_Production_Rank':range(1,23,1),'AC_Production_Rank':ac_pr_per_dc_rank_p2})
plant_2_dc_ac_rank

# Is there any missing data?

In [ ]:
print("*******************Plant_1_Sensor*****************")
print(plant1_sensor.isna().sum()/plant1_sensor.shape[0]*100)
print("*******************Plant_1_Generation*****************")
print(plant1_generation.isna().sum()/plant1_generation.shape[0]*100)
print("*******************Plant_2_Sensor*****************")
print(plant2_sensor.isna().sum()/plant2_sensor.shape[0]*100)
print("*******************Plant_2_Generation*****************")
print(plant2_generation.isna().sum()/plant2_generation.shape[0]*100)